# Logistic Regression in scikit-learn and PyTorch

In this notebook, we are going to implement a simple multi-class logistic regression in both scikit-learn and PyTorch.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. The Data

We are going to use the Iris data set - a 150 instance, 3-class data set with 4 features.

For now, we won't bother with the standard train/test splits - we just want a model that works.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
print(iris.keys())

In [ ]:
print(iris.data.shape)
print(iris.target.shape)

In [ ]:
iris.data[:5]

In [ ]:
iris.target

Note that our target $y$ is encoded as a single array with classes as its values.

## 2. Logistic Regression in scikit-learn

We will use the built-in `LogisticRegression` classifier in scikit-learn. To make this implementation consistent with our PyTorch formulation, we will slightly modify the defaults - namely modifying the multi-class calculation to use a softmax, and turning off the regularization.

(If you do not know what 'regularization' means, it will be covered in a later lecture. For now, just know it is an additional part of the model we are not concerned with.)

First, we set up the model with our desired arguments.

In [ ]:
from sklearn.linear_model import LogisticRegression
sk_model = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=1000000,
    C=np.finfo(np.float).max,
)

Next, we fit the model to the data

In [ ]:
sk_model.fit(X=iris.data, y=iris.target)

Finally, we can predict probabilities that each sample is in each class. Note that the probabilities here will be *very* high, because we are clearly overfitting to the data set.

In [ ]:
sk_model.predict_proba(X=iris.data)[:5]

## 3. Logistic Regression in PyTorch

First, we will import some modules from PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Next, we will implement a model in PyTorch -- as an `nn.Module`.

A `nn.Module` can really be any function, but it is often used to implement layers, functions and models. Note that you can also nest modules.

Importantly, modules need to have their `forward()` method overridden, and very often you will want to override the `__init__` method as well. 

The `__init__` method sets up the module, akin to how we set up the `LogisticRegression` model above with some arguments. This is also often where the internal modules and parameters are initialized.

The `forward` method defines what happens when you *apply* the module.

In the background, PyTorch makes use of your code in the forward method and determines how to implement back-propagation with it - but all you need to do is to define the forward pass!

As a reminder, softmax 

In [ ]:
class LogisticRegressionPyTorch(nn.Module):
    def __init__(self, n_in, n_out):
        """
        n_in: Number of features
        n_out: Number of output classes
        """
        # Initialize the parent class - this is a Python requirement
        super().__init__()
        
        # Set up out linear layer. This initializes the weights
        # Note that self.linear is itself a nn.Module, nested within
        #   this module
        self.linear = nn.Linear(n_in, n_out)
        
        # Explicitly initialize the weights with the initialization
        #   scheme we want.
        self.init_weights()
        
    def forward(self, x):
        """
        x: Input data [N, k]
        ---
        Returns: log probabilities of each class [N, c]
        """
        # Apply the linear function to get our logit (real numbers)
        logit = self.linear(x)
        
        # Apply log_softmax to get logs of normalized probabilities
        return F.log_softmax(logit, dim=1)
    
    def init_weights(self):
        # Use some specific initialization schemes
        nn.init.xavier_normal_(self.linear.weight)
        nn.init.uniform_(self.linear.bias)

In [ ]:
x = torch.from_numpy(iris.data).float()
y = torch.from_numpy(iris.target)

In [ ]:
# List to record our loss over time
loss_val_ls = []

# Initialize our model. Note we need to provide n_in and n_out
pt_model = LogisticRegressionPyTorch(n_in=x.shape[1], n_out=3)

# Set up our criterion - our loss function
criterion = nn.NLLLoss()

# Set up our optimizer. We need to tell the optimizer which
#   parameters it will optimize over (which parameters it is
#   allowed to modify).
optimizer = optim.Adam(pt_model.parameters(), lr=0.1)

In [ ]:
# As a quick check, lets investigate the number of parameters in our model:
for param in pt_model.parameters():
    print(param.shape)

In [ ]:
# Run over many iterations!
for i in range(10000):
    
    # Always zero-out the gradients managed by your optimizer
    # PyTorch does not automatically zero-out your gradients
    #   You can also do pt_model.zero_grad() in this case.
    optimizer.zero_grad()
    
    # !! Put model into training mode. This does not do anything
    #   in a simple Logistic Regression model, but will be important 
    #   later. (See: Dropout)
    pt_model.train()
    
    # Compute the predicted log-probabilities
    y_hat = pt_model(x)
    
    # Compute the loss
    train_loss = criterion(y_hat, y)
    
    # Back-propagate the gradients to the parameters
    train_loss.backward()
    
    # Apply the gradient updates to the parameters
    optimizer.step()
    
    # Recompute the loss in evaluation mode, and record it.
    # Again, this does not do anything here, but will be important later.
    # Since we are evaluating, we will also tell PyTorch not to
    #   compute gradients.
    pt_model.eval()
    with torch.no_grad():
        y_hat = pt_model(x)
        eval_loss = criterion(y_hat, y)
        
    # Record the loss
    # Note that 'loss' is a Tensor, but loss.item() is a number
    loss_val_ls.append(eval_loss.item())

In [ ]:
plt.plot(loss_val_ls)
plt.xlabel("Epochs")
plt.ylabel("Log Loss")

In [ ]:
# See our final loss
loss_val_ls[-1]

In [ ]:
# To confirm that our model is doing the right thing, we do a quick test.
# We create a new logistic regression model in PyTorch, but we
#   insert the weights learned from our scikit-learn model.
# We compute the loss similarly.
# Note that it is numerically similar to the loss above!
# (See if you can understand what every line is doing.)
with torch.no_grad():
    blank_model = LogisticRegressionPyTorch(n_in=x.shape[1], n_out=3)
    blank_model.linear.weight.set_(
        torch.from_numpy(sk_model.coef_).float()
    )
    blank_model.linear.bias.set_(
        torch.from_numpy(sk_model.intercept_).float()
    )
    y_hat = blank_model(x)
    print(criterion(y_hat, y).item())

## 4. Questions for yourself

1. We asserted that the models are roughly equivalent because they reached the same losses. But is this true? Can we directly compare the parameter values? (Try it!) What if the parameter values are different?

2. In scikit-learn, you can use `.predict_proba` to compute the predicted probabilities. How do we do the same for our PyTorch model?

3. Although we showed that the loss is numerically the same, and you can confirm for yourself that the predictions $\hat{y}$ are numerically similar between the scikit-learn and PyTorch implementations, if you inspect the actual weights and biases, you will notice that they are different. Why is this the case?

## 5. Things you should look up

1. How to perform training on GPUs
    * Hint: both model and training data need to be on GPU
2. How to incorporate regularization
    * Plus: How to record the regularization loss (i.e. not in optimizer)
3. How to save / load models
    * Hint: `torch.save`, `torch.load`, and use `model.state_dict()`
4. The difference between, e.g. `nn.LogSoftmax()` and `F.log_softmax()`
5. The difference between `nn.NLLLoss`, and `nn.CrossEntropyLoss`, and when to use each one

**If anything is unclear, please come to office hours!**